# Generate noisy signals

The default Langmuir samples are not noisy and, thus, do not properly reflect real world data.  This notebook will add noise to the signals to help simulate real world data.

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os

from plasmapy.utils.decorators import validate_quantities

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

In [ ]:
# noise generator
@validate_quantities
def add_noise(signal, snr_db: u.dB = 10 * u.dB):
    """
    Add noise to a signal based on a signal-to-noise ratio (SNR).

    .. math::

        SNR &= \\frac{P_{signal}}{P_{noise}}

        SNR_{dB} &= 10 \\log_{10}(SNR)

    where :math:`P_{signal}` and :math:`P_{noise}` is the average power of
    signal and noise components, respectively.

    The noise component is generated using `numpy.random.normal`.

    Parameters
    ----------
    signal: array_like
        1D array to add noise to

    snr_db: `~astropy.units.Quantity`
        desired signal-to-noise ratio in decibels

    Returns
    -------
    array_like
        the original signal with noise added to it

    References
    ---------
    .. [1] https://en.wikipedia.org/wiki/Signal-to-noise_ratio
    .. [2] https://stackoverflow.com/a/53688043

    """
    sig_pwr = signal**2
    sig_pwr_ave = np.mean(sig_pwr)
    sig_pwr_ave_db = 10.0 * np.log10(sig_pwr_ave)

    noise_pwr_ave_db = sig_pwr_ave_db - snr_db.value
    noise_pwr_ave = 10.0 ** (noise_pwr_ave_db / 10.0)

    noise = np.random.normal(0.0, np.sqrt(noise_pwr_ave), signal.size)

    return signal + noise

In [ ]:
# read in example data
# this can be found at
#   https://github.com/PlasmaPy/PlasmaPy/blob/main/docs/notebooks/langmuir_samples/Beckers2017.npy
#
filename = "Beckers2017"
voltage, current = np.load(filename + ".npy")


# add some artificial noise to simulate reliztic digitized signals
v_noisy = add_noise(voltage, 38 * u.dB)
i_noisy = add_noise(current, 28 * u.dB)

In [ ]:
figwidth, figheight = plt.rcParams["figure.figsize"]
figheight = 2.0 * figheight
fig, axs = plt.subplots(3, 1, figsize=[figwidth, figheight])

# Current plot
axs[0].set_ylabel("Current", fontsize=12)
axs[0].set_xlabel("Index", fontsize=12)
axs[0].plot(i_noisy, label="noisy")
axs[0].plot(current, color="r", label="original")
axs[0].legend(fontsize=12)

# Voltage plot
axs[1].set_ylabel("Voltage", fontsize=12)
axs[1].set_xlabel("Index", fontsize=12)
axs[1].plot(v_noisy, label="noisy")
axs[1].plot(voltage, color="r", label="original")
axs[1].legend(fontsize=12)

# IV plot
axs[2].set_ylabel("Current", fontsize=12)
axs[2].set_xlabel("Voltage", fontsize=12)
axs[2].plot(v_noisy, i_noisy, label="noisy")
axs[2].plot(voltage, current, color="r", label="original")
axs[2].legend(fontsize=12)

In [ ]:
save = False
if save:
    savefile = filename + "_noisy.npy"
    with open(savefile, "wb") as fp:
        np.save(fp, [v_noisy, i_noisy])